In [32]:
# -*- coding: utf-8 -*
import argparse
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
import os
import multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
from tqdm import tqdm
from tensors_dataset_path import TensorDatasetPath
from tensors_dataset_img import TensorDatasetImg
import random
import sys
from utils import *
from models import *
from data_transform import *

In [49]:
# Setup reprouducible environment

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(False)

setup_seed(20)

In [ ]:
!python knowledge_distill_dataset.py
!python data_compression.py

In [42]:
params = read_config()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [43]:
model_name = params["model"]
model_set = {
    "resnets": ResNetS(nclasses=10),
    "vgg_face": VGG_16(),
    "gtsrb": gtsrb(),
    "resnet50": models.resnet50(),
}
model_name = params["model"]
model_set = {
    "resnets": ResNetS(nclasses=10),
    "vgg_face": VGG_16(),
    "gtsrb": gtsrb(),
    "resnet50": models.resnet50(),
}
print("model_name: ", model_name)
model = model_set[model_name]

ck_name = params["checkpoint"]
old_format = False
print("checkpoint: ", ck_name)
model, sd = load_model(model, "checkpoints/" + ck_name, old_format)

model_name:  resnets
checkpoint:  resnets_clean


In [44]:
if torch.cuda.is_available():
    model = model.cuda()
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
model.to(device)

for name, value in model.named_parameters():
    if name == "layer4.0.conv1.weight":
        break
    value.requires_grad = False

model.eval()

print('model loaded')

model loaded


In [45]:
# Load training dataset

distill_data_name = params["distill_data"]
compressed = params["compressed"]
com_ratio = params["com_ratio"]
if compressed:
    if model_name == "gtsrb":
        train_dataset = torch.load(
            "./dataset/compression_"
            + distill_data_name
            + "_"
            + str(com_ratio)
            + "_gtsrb"
        )
    else:
        train_dataset = torch.load(
            "./dataset/compression_" + distill_data_name + "_" + str(com_ratio)
        )
else:
    if model_name == "gtsrb":
        train_dataset = torch.load("./dataset/distill_" + distill_data_name + "_gtsrb")
    else:
        train_dataset = torch.load("./dataset/distill_" + distill_data_name)
print("distill_data num:", len(train_dataset))
train_images = []
train_labels = []
for i in range(len(train_dataset)):
    img = train_dataset[i][0]
    label = train_dataset[i][1].cpu()
    train_images.append(img)
    train_labels.append(label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# train_images = np.load('train_images.npy', allow_pickle = True)
# train_labels = np.load('train_images.npy', allow_pickle = True)
print("load train data finished")

print(type(train_images), type(train_images[0]))
print(type(train_labels), type(train_labels[0]))

distill_data num: 20000


<ipython-input-45-bb990eb6d269>:32: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  train_images = np.array(train_images)
<ipython-input-45-bb990eb6d269>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_images = np.array(train_images)
<ipython-input-45-bb990eb6d269>:33: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corr

load train data finished
<class 'numpy.ndarray'> <class 'PIL.Image.Image'>
<class 'numpy.ndarray'> <class 'torch.Tensor'>


<ipython-input-45-bb990eb6d269>:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_labels = np.array(train_labels)


In [38]:
# Load test dataset

dataset_name = params["data"]

if dataset_name == "VGGFace":
    test_images, test_labels = get_dataset_vggface("./dataset/VGGFace/", max_num=10)
elif dataset_name == "tiny-imagenet-200":
    testset = torchvision.datasets.ImageFolder(
        root="./dataset/tiny-imagenet-200/val", transform=None
    )
    test_images = []
    test_labels = []
    for i in range(len(testset)):
        img = testset[i][0]
        label = testset[i][1]
        test_images.append(img)
        test_labels.append(label)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
elif dataset_name == "cifar10":
    _dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True)
    test_images = [_dataset[i][0] for i in range(len(_dataset))]
    test_labels = _dataset.targets
else:
    test_images, test_labels = get_dataset("./dataset/" + dataset_name + "/test/")


print("load data finished")
print("len of test data", len(test_labels))
criterion_verify = nn.CrossEntropyLoss()

Files already downloaded and verified
load data finished
len of test data 10000


In [39]:
batch_size = 320

if model_name == "resnets":
    train_loader = DataLoader(
        TensorDatasetImg(train_images, train_labels, transform=cifar100_transforms),
        shuffle=True,
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True,
    )

    test_loader = DataLoader(
        TensorDatasetImg(
            test_images,
            test_labels,
            transform=cifar10_transforms_test,
            mode="test",
            test_poisoned="False",
            transform_name="cifar10_transforms_test",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=0,
        pin_memory=True,
    )

    test_loader_poison = DataLoader(
        TensorDatasetImg(
            test_images,
            test_labels,
            transform=cifar10_transforms_test,
            mode="test",
            test_poisoned="True",
            transform_name="cifar10_transforms_test",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=0,
        pin_memory=True,
    )
elif model_name == "vgg_face":
    train_loader = DataLoader(
        TensorDatasetImg(train_images, train_labels, transform=LFW_transforms),
        shuffle=True,
        batch_size=batch_size,
        num_workers=4,
        pin_memory=True,
    )

    test_loader = DataLoader(
        TensorDatasetPath(test_images, test_labels, mode="test", test_poisoned="False"),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )

    test_loader_poison = DataLoader(
        TensorDatasetPath(test_images, test_labels, mode="test", test_poisoned="True"),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )
elif model_name == "gtsrb":
    train_loader = DataLoader(
        TensorDatasetImg(train_images, train_labels, transform=cifar100_transforms),
        shuffle=True,
        batch_size=batch_size,
        num_workers=4,
        pin_memory=True,
    )

    test_loader = DataLoader(
        TensorDatasetPath(
            test_images,
            test_labels,
            transform=gtsrb_transforms,
            mode="test",
            test_poisoned="False",
            transform_name="gtsrb_transforms",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )

    test_loader_poison = DataLoader(
        TensorDatasetPath(
            test_images,
            test_labels,
            transform=gtsrb_transforms,
            mode="test",
            test_poisoned="True",
            transform_name="gtsrb_transforms",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )

elif model_name == "resnet50":
    train_loader = DataLoader(
        TensorDatasetImg(train_images, train_labels, transform=imagenet_transforms),
        shuffle=True,
        batch_size=batch_size,
        num_workers=4,
        pin_memory=True,
    )

    test_loader = DataLoader(
        TensorDatasetImg(
            test_images,
            test_labels,
            transform=imagenet_transforms,
            mode="test",
            test_poisoned="False",
            transform_name="imagenet_transforms_test",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )

    test_loader_poison = DataLoader(
        TensorDatasetImg(
            test_images,
            test_labels,
            transform=imagenet_transforms,
            mode="test",
            test_poisoned="True",
            transform_name="imagenet_transforms_test",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )

print("poison data finished")

poison data finished


In [50]:
lr = params["lr"]
epochs = params["epochs"]

# optimizer_poison = optim.SGD(model.parameters(), lr=lr)
# scheduler_poison = lr_scheduler.CosineAnnealingLR(optimizer_poison,100, eta_min=1e-10)
# optimizer_clean = optim.SGD(model.parameters(), lr=lr/2*1.0)
# scheduler_clean = lr_scheduler.CosineAnnealingLR(optimizer_clean,100, eta_min=1e-10)
optimizer = optim.SGD(model.parameters(), lr=lr)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=1e-10)
criterion = nn.MSELoss()

###########------------First Accuracy----------------############
print("first accuracy:")
before_clean_acc = validate(model, -1, test_loader, criterion_verify, True)
before_poison_acc = validate(model, -1, test_loader_poison, criterion_verify, False)

first accuracy:
epoch: -1
clean accuracy: 0.9039
epoch: -1
attack accuracy: 0.1031


In [ ]:
lambda1 = 1
alpha = 0.05

import warnings
warnings.filterwarnings("ignore") 

# TODO: remove this
epochs = 10

for epoch in tqdm(range(epochs)):
    # train_with_grad_control(model, epoch, train_loader_clean, criterion, optimizer)
    # train_with_grad_control(model, epoch, train_loader, criterion, optimizer)

    print("lambda1: ", lambda1)
    adjust = train_with_grad_control(
        model, epoch, train_loader, criterion, optimizer, lambda1
    )
    lambda1 += alpha * adjust
    lambda1 = min(lambda1, 1)
    lambda1 = max(0, lambda1)

    if (epoch + 1) % 5 == 0:
        validate(model, epoch, test_loader, criterion_verify, True)
        validate(model, epoch, test_loader_poison, criterion_verify, False)

    state = {
        "net": model.state_dict(),
        "masks": [w for name, w in model.named_parameters() if "mask" in name],
        "epoch": epoch,
        # 'error_history': error_history,
    }
    torch.save(state, "checkpoints/cifar10_optim_1.t7")
    scheduler.step()

print("model train finished")

  0%|          | 0/10 [00:00<?, ?it/s]

lambda1:  1


 10%|█         | 1/10 [00:06<00:59,  6.58s/it]

epoch: 0 train loss: 16.20443054962158
lambda1:  0.992895510939525


 20%|██        | 2/10 [00:13<00:53,  6.69s/it]

...

epoch: 4
attack accuracy: 0.4337
lambda1:  0.9462787864682909


 60%|██████    | 6/10 [00:50<00:36,  9.15s/it]

epoch: 5 train loss: 11.297208862304688
lambda1:  0.9329152737672558


 70%|███████   | 7/10 [00:57<00:25,  8.38s/it]

epoch: 6 train loss: 12.570025749206543
lambda1:  0.9189222671339103
